# Import necessary lib. 


In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.contrib.layers import l1_l2_regularizer

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)

import os
import h5py
import time
# from batchnorm import batch_norm,fc_batch_norm

from sklearn.metrics import accuracy_score,classification_report
import Model_1226 as my_model

E:\Liulicong\Software\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.12.0


# Configuration
Set the class names and num (17).
Set the data path (relative path). 

In [2]:
# configuration
# data attribution
CLASS_NUM = 17
CLASS_URBAN = ['Urban_Compact high-rise', 'Urban_Compact mid-rise', 'Urban_Compact low-rise',
               'Urban_Open high-rise', 'Urban_Open mid-rise', 'Urban_Open low-rise',
               'Urban_Lightweight low-rise', 'Large low-rise', 'Sparesly built', 'Heavy industry']
CLASS_RURAL = ['Dense trees', 'Scattered trees', 'Bush and scrub', 'Low plants', 'Bare rock or paved', 'Bare soil or sand', 'Water']
CLASS_NAME = CLASS_URBAN + CLASS_RURAL

# data path
base_dir = r'../data/'
path_training = os.path.join(base_dir, 'training.h5')
path_validation = os.path.join(base_dir, 'validation.h5')
path_test = os.path.join(base_dir, 'round1_test_a_20181109.h5')
# random
np.random.seed(0)

# Read Data
Check the data foramt.

In [3]:
h5py_training = h5py.File(path_training,'r')
h5py_validation = h5py.File(path_validation,'r')
h5py_test = h5py.File(path_test, 'r')

### get s1 image channel data
### it is not really loaded into memory. only the indexes have been loaded.
print("-" * 60)
print("training part")
s1_training = h5py_training['sen1']
print('s1_training: ', s1_training.shape)
s2_training = h5py_training['sen2']
print('s2_training: ', s2_training.shape)
label_training = h5py_training['label']
print('label_training: ', label_training.shape)

print("-" * 60)
print("validation part")
s1_validation = h5py_validation['sen1']
print('s1_validation: ', s1_validation.shape)
s2_validation = h5py_validation['sen2']
print('s2_validation: ', s2_validation.shape)
label_validation = h5py_validation['label']
print('label_validation: ', label_validation.shape)

print("-" * 60)
print("Test part")
s1_test = h5py_test['sen1']
print('s1_test: ', s1_test.shape)
s2_test = h5py_test['sen2']
print('s2_test: ', s2_test.shape)


train_num, pixel_x, pixel_y, sar_bands = s1_training.shape
vali_num, pixel_x, pixel_y, optical_bands = s2_validation.shape
test_num = s1_test.shape[0]

------------------------------------------------------------
training part
s1_training:  (352366, 32, 32, 8)
s2_training:  (352366, 32, 32, 10)
label_training:  (352366, 17)
------------------------------------------------------------
validation part
s1_validation:  (24119, 32, 32, 8)
s2_validation:  (24119, 32, 32, 10)
label_validation:  (24119, 17)
------------------------------------------------------------
Test part
s1_test:  (4838, 32, 32, 8)
s2_test:  (4838, 32, 32, 10)


# CV Training

In [4]:
MODEL_Prefix = "10CV_baseline"
N = 10

In [5]:
from sklearn.model_selection import StratifiedKFold
sfolder = StratifiedKFold(n_splits=N, random_state=86, shuffle=False)

In [6]:
for i, (train_loc, vali_loc) in enumerate(sfolder.split(np.zeros((vali_num, 1)), np.zeros(vali_num))):
    print('Train: %s | test: %s' % (train_loc, vali_loc))
    
    X_optical_train = s2_validation[train_loc, :, :, :]
    y_train = label_validation[train_loc, :]
    
    X_optical_vali = s2_validation[vali_loc, :, :, :]
    y_vali = label_validation[vali_loc, :]
    
    my_model.build_model("{}{}".format(MODEL_Prefix, i))
    my_model.training((X_optical_train, y_train), (X_optical_vali, y_vali), model_name="{}{}".format(MODEL_Prefix, i), epochs=20)

Train: [ 2412  2413  2414 ... 24116 24117 24118] | test: [   0    1    2 ... 2409 2410 2411]
Model Build Start!
Model Build Success!
start training!
Loss change at:  0 With AC:  0.09230769282350174 With Loss:  2.818756008622658
Loss change at:  450 With AC:  0.4873076929495885 With Loss:  1.9490005222719107
Loss change at:  500 With AC:  0.5966666684700892 With Loss:  1.3165710118674916
Loss change at:  550 With AC:  0.6255128269012158 With Loss:  1.2101841241763795
Loss change at:  600 With AC:  0.7839743586686941 With Loss:  0.6057926888884992
Loss change at:  700 With AC:  0.823846147610591 With Loss:  0.5232774088434121
Loss change at:  750 With AC:  0.8349999968822186 With Loss:  0.4738428311956858
Loss change at:  850 With AC:  0.8646153807640076 With Loss:  0.38873380887172315
Loss change at:  950 With AC:  0.8828205099472632 With Loss:  0.3805735544185733
Loss change at:  1000 With AC:  0.8725640911322373 With Loss:  0.37616273617467677
Loss change at:  1150 With AC:  0.8847435

Loss change at:  1450 With AC:  0.9226923172290509 With Loss:  0.26173789991964747
Loss change at:  1850 With AC:  0.9242307681303757 With Loss:  0.2394103025816764
Loss change at:  2050 With AC:  0.93000001173753 With Loss:  0.23473285111424144
Loss change at:  2150 With AC:  0.9300000071525574 With Loss:  0.22156427542417995
Time use:  102.5658049583435
Train: [    0     1     2 ... 24116 24117 24118] | test: [14472 14473 14474 ... 16881 16882 16883]
Model Build Start!
Model Build Success!
start training!
Loss change at:  0 With AC:  0.11897436013588539 With Loss:  2.851729484735239
Loss change at:  450 With AC:  0.38243589951441836 With Loss:  2.4149696403870338
Loss change at:  500 With AC:  0.6217948702665476 With Loss:  1.4391237847247527
Loss change at:  550 With AC:  0.74756409571721 With Loss:  0.7645365151995253
Loss change at:  600 With AC:  0.8170512822958139 With Loss:  0.5239312692265803
Loss change at:  650 With AC:  0.8401282062897315 With Loss:  0.46760346086859506
Los

# CV Test 

In [7]:
CV_proba = np.zeros((test_num, CLASS_NUM))
for i in range(N):
    my_model.build_model("{}{}".format(MODEL_Prefix, i))
    CV_proba += my_model.batch_predict(s2_test, "{}{}".format(MODEL_Prefix, i))

Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ckpt/10CV_baseline0.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ckpt/10CV_baseline1.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ckpt/10CV_baseline2.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ckpt/10CV_baseline3.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ckpt/10CV_baseline4.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ckpt/10CV_baseline5.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ckpt/10CV_baseline6.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ckpt/10CV_baseline7.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ckpt/10CV_baseline8.ckpt
Model Build Start!
Model Build Succes

In [8]:
CV_proba_max_loc = np.argmax(CV_proba, 1)
one_hot=tf.one_hot(CV_proba_max_loc,CLASS_NUM)
with tf.Session() as sess:
    _one_hot = sess.run(one_hot)
_one_hot = _one_hot.astype(np.int16)
pd.DataFrame(_one_hot).to_csv('1227_deep_result_10CV{}.csv'.format(N), index=False, header=None)